In [13]:
import numpy as np
import pandas as pd

import sklearn.preprocessing as skpreprocessing
import sklearn.manifold as skmanifold
import sklearn.decomposition as skdecomposition

from joblib import Parallel, delayed

# Global Settings

In [14]:
global maxSamples

processes     = 3  # Set the number of parallel processes to use

maxSamples = 20000  # Maximum number of samples to use

performTSNE   = True  # if True then perform t-SNE, if False don't
performPCA    = True  # if True then perform PCA, if False don't
performISOMAP = True  # if True then perform isoMap, if False don't

# Set low mass limits for clouds
lowLim512  = 8.  # Still tbd
lowLim1024 = 4.  # Still tbd
lowLim2048 = 1.

# Choose which columns we're interested in, i.e. which 'features' we want to investigate
featureColumns = ['logRPosition','vMag', 'polarAngle', 'logMass', 'time']

# Import and setup Dataframes

In [3]:
# Load the datasets and print headers
catalog512 = np.load('../data/physCatalog512.npy')
catalog1024 = np.load('../data/physCatalog1024.npy')
catalog2048 = np.load('../data/physCatalog2048.npy')

In [4]:
# Convert to dataframes with mass, volume, radial distance, and magnitude of velocity

# ==============================================================================
def converter(catalog):
    # Copy data to the dataFrame
    outputDF = pd.DataFrame()
    
    outputDF['ID']         = catalog['ID']
    outputDF['volume']     = catalog['volume']
    outputDF['mass']       = catalog['mass']
    outputDF['logMass']    = np.log10(catalog['mass'])
    outputDF['rPosition']  = np.sqrt(catalog['positionX']**2 
                                     + catalog['positionY']**2 
                                     + catalog['positionZ']**2)
    outputDF['logRPosition'] = np.log10(outputDF['rPosition'])
    outputDF['zPosition']  = catalog['positionZ'].reshape(-1, 1)
    outputDF['vMag']       = np.sqrt(catalog['velocityX']**2 
                                     + catalog['velocityY']**2 
                                     + catalog['velocityZ']**2)
    outputDF['polarAngle'] = np.arccos(np.abs(outputDF['zPosition'])/outputDF['rPosition'])
    outputDF['resolution'] = catalog['resolution']
    outputDF['time']       = catalog['time']

    return outputDF
# ==============================================================================

processed512  = converter(catalog512)
processed1024 = converter(catalog1024)
processed2048 = converter(catalog2048)

In [5]:
# ==============================================================================
def chooseSubset(catalog, lowLim):
    # Cut out the low mass clouds that are poorly sampled
    catalog = catalog[catalog['mass'] > lowLim]

    if maxSamples > catalog.shape[0]:
        numRows = catalog.shape[0]
    else:
        numRows = maxSamples
    
    print(f'{numRows = } for resolution = {catalog["resolution"].iloc[1]}')

    return catalog.sample(n=numRows, replace=False)
# ==============================================================================

# Now call the function with the chosen catalog
subset512  = chooseSubset(processed512,  lowLim512)
subset1024 = chooseSubset(processed1024, lowLim1024)
subset2048 = chooseSubset(processed2048, lowLim2048)

# Create data only arrays for learning and rescale them
subset512Data  = subset512[featureColumns].values
subset1024Data = subset1024[featureColumns].values
subset2048Data = subset2048[featureColumns].values

# Rescale data
scaler = skpreprocessing.StandardScaler(with_mean=True, with_std=True)

subset512Data  = scaler.fit_transform(subset512Data)
subset1024Data = scaler.fit_transform(subset1024Data)
subset2048Data = scaler.fit_transform(subset2048Data)

dataList = [subset512Data, subset1024Data, subset2048Data]

numRows = 17268 for resolution = 512
numRows = 20000 for resolution = 1024
numRows = 20000 for resolution = 2048


# Dimensionality Reduction

## t-SNE

t-SNE (t-distributed Stochastic Neighbor Embedding) is a *non-linear*  dimensionality reduction algorithm. It's good at dealing with non-linear data but takes a long time to run and is mostly used for visualization.

In [6]:
# ==============================================================================
def runTSNE(catalog, perp):
    """
    Run t-SNE dimensionality reduction on the data and return the arrays for plotting
    """
    # Create the model
    tsneModel   = skmanifold.TSNE(n_components=2, 
                                        perplexity=perp, 
                                        init='pca')
    
    # Perform the dimensionality reduction
    tsneResults = tsneModel.fit_transform(catalog)

    return (tsneResults[:,0], tsneResults[:,1])
# ==============================================================================

In [7]:
if performTSNE:

    # perplexity should be between 5 and 100, analogous to number of neareset neighbors
    # I've found that 100 seems to work best
    perp = 100
    
    # Do the dimensionality reduction and generating figures
    tSNEColumns = Parallel(n_jobs=processes)(delayed(runTSNE)
                                            (tsneCatalog, perp)
                                            for tsneCatalog in dataList)

    # Copy the returned data to the respective dataframes
    subset512['tsne-0']  = tSNEColumns[0][0]
    subset512['tsne-1']  = tSNEColumns[0][1]
    subset1024['tsne-0'] = tSNEColumns[1][0]
    subset1024['tsne-1'] = tSNEColumns[1][1]
    subset2048['tsne-0'] = tSNEColumns[2][0]
    subset2048['tsne-1'] = tSNEColumns[2][1]
    
    # Remove unused variable
    del tSNEColumns

## PCA

PCA (Principal Component Analysis) is a method *linear* method of dimensionality reduction. Typically it uses SVD (Singular Value Decomposition)to do this and it can be used to either speed up machine learning or visualize data. PCA is fast and effective for linear data but totally fails on non-linear data.

In [8]:
# ==============================================================================
def runPCA(catalog):
    """
    Run PCA dimensionality reduction on the data and return the arrays for plotting
    """
    # Create the model
    pcaModel   = skdecomposition.PCA(n_components=2)
    
    # Perform the dimensionality reduction
    pcaResults = pcaModel.fit_transform(catalog)

    return (pcaResults[:,0], pcaResults[:,1])
# ==============================================================================

In [9]:
if performPCA:
    # Do the dimensionality reduction and generating figures
    pcaColumns = Parallel(n_jobs=processes)(delayed(runPCA)
                                            (pcaCatalog)
                                            for pcaCatalog in dataList)

    # Copy the returned data to the respective dataframes
    subset512 ['pca-0']  = pcaColumns[0][0]
    subset512 ['pca-1']  = pcaColumns[0][1]
    subset1024['pca-0']  = pcaColumns[1][0]
    subset1024['pca-1']  = pcaColumns[1][1]
    subset2048['pca-0']  = pcaColumns[2][0]
    subset2048['pca-1']  = pcaColumns[2][1]
    
    # Remove unused variable
    del pcaColumns

## Random Projection

Preserves the Euclidean distance between points while embedding high-dimensional values. Runs much faster than PCA on data sets with a large number of samples. However it doesn't always do a good job of reducing dimensionality from a high number (say 1,000) to a low number (2-3). The limit on this is given by

$$
    D \geq 4 \frac{\epsilon^2}{\left(2-\epsilon^3\right) / 3}^{-1} \log{M}
$$

Where $D$ is the number of dimension, $\epsilon$ is the error rate (between 0 and 1), and $M$ is the number of samples.

Which due to our low number of dimensions and high number of data points is always much larger than 2 (usually several hundred).

## ISOMAP

Isometric Mapping (ISOMAP) is a manifold learning method that uses geodesic distances among the data points. It is *non-linear*. It uses KNN to find the geodesic then a path finding algorithm to determine the distance. Can be computationally expensive.


[more details](https://medium.com/data-science-in-your-pocket/dimension-reduction-using-isomap-72ead0411dec)

In [10]:
# ==============================================================================
def runISOMAP(catalog):
    """
    Run ISOMAP dimensionality reduction on the data and return the arrays for plotting
    """
    # Create the model
    isomapModel   = skmanifold.Isomap(n_components=2)
    
    # Perform the dimensionality reduction
    isomapResults = isomapModel.fit_transform(catalog)

    return (isomapResults[:,0], isomapResults[:,1])
# ==============================================================================

In [11]:
if performISOMAP:
    # Do the dimensionality reduction and generating figures
    isomapColumns = Parallel(n_jobs=processes)(delayed(runPCA)
                                            (pcaCatalog)
                                            for pcaCatalog in dataList)

    # Copy the returned data to the respective dataframes
    subset512 ['isomap-0']  = isomapColumns[0][0]
    subset512 ['isomap-1']  = isomapColumns[0][1]
    subset1024['isomap-0']  = isomapColumns[1][0]
    subset1024['isomap-1']  = isomapColumns[1][1]
    subset2048['isomap-0']  = isomapColumns[2][0]
    subset2048['isomap-1']  = isomapColumns[2][1]
    
    # Remove unused variable
    del isomapColumns

# Saving to Disk

In [12]:
savePath  = "../data/"
extension = ".pkl"

subset512.to_pickle(savePath + 'dimReduceCatalog512' + extension)
subset1024.to_pickle(savePath + 'dimReduceCatalog1024' + extension)
subset2048.to_pickle(savePath + 'dimReduceCatalog2048' + extension)